In [1]:
import SimpleITK as sitk
import pandas as pd
import os
import numpy as np
import matplotlib.colors as mcolors
import csv

In [2]:
#
# open structure properties file
#
csv_file = 'structure_properties.csv'
df = pd.read_csv(csv_file)
df.set_index('acronym',inplace=True)

In [3]:
# output directory
output_directory = 'uci_ad_ccf'

In [4]:
#
# open annotation file
#
model_directory = "C:/Users/lydia/OneDrive/Work/DigitalAssets/mouse_ccf"
annotation_file = os.path.join( model_directory,  "annotation/ccf_2017", "annotation_10.nrrd")

annotation = sitk.ReadImage( annotation_file )

In [5]:
def create_mask( structure, annotation, df, fg = 1 ) :
    
    slist = df.loc[structure]['substructures']
    slist = slist.split('/')
    slist = [ int(x) for x in slist ]
    
    arr = sitk.GetArrayFromImage( annotation )
    idx = np.isin(arr,slist)
    output = np.zeros(arr.shape,np.uint16)
    output[idx] = fg
    
    output_image = sitk.GetImageFromArray( output )
    output_image.CopyInformation( annotation )    
    return output_image

In [6]:
mask = create_mask('VL', annotation, df )

In [7]:
output_file = os.path.join(output_directory, 'VL_mask_image.nrrd')
sitk.WriteImage( mask, output_file, True )

In [12]:
#
# open updated annotation file
#
model_directory = "C:/Users/lydia/OneDrive/Work/DigitalAssets/UpdatedCCF_2020"
annotation_file = os.path.join( model_directory, "2020_updated_annotation_10.nrrd")

annotation = sitk.ReadImage( annotation_file )

In [13]:
mask = create_mask('CA', annotation, df)
mask = sitk.Not(mask)

In [14]:
output = sitk.Mask( annotation, mask, 0, 1 )

In [15]:
output_file = os.path.join(output_directory, 'CA_layer_annotation.nrrd')
sitk.WriteImage( output, output_file, True )